[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brunomssmelo/Python_em_Auditoria/blob/master/06_Estratificacao.ipynb)

# **Análise de Dados aplicada à Auditoria com _Python_**

## **Técnicas de Auditoria Assistidas por Computador - TAACs**
### _Estratificação_

In [1]:
# Antes de prosseguirmos, deveremos carregar as bibliotecas necessárias:
import pandas as pd #pandas

In [2]:
# A importação dos dados pode ser realizada com o método read_csv():
faturas = pd.read_csv('./dados/Invoices.csv',  encoding='latin1', sep=';', decimal=',', na_values=' ')

# Podemos visualizar os registros iniciais dos dados importados com o método head():
faturas.head()

,Date,InvoiceNo,CustomerNo,SalesPerson,ProductNo,UnitPrice,Quantity,Amount
0,09/07/2003,20000,10220,8,8,9.2,41,377.2
1,21/08/2003,20001,10491,4,48,14.0,30,420.0
2,27/08/2003,20002,10704,3,43,15.0,25,375.0
3,28/05/2003,20003,10430,5,54,24.0,22,528.0
4,06/12/2003,20004,10841,17,11,15.0,21,315.0


<br>

A estratificação consiste em classificar os registros de um conjunto de dados em estratos mutuamente exclusivos.

Em auditoria esta classificação é feita usualmente com base em valores monetários. O procedimento consiste em se 
estabelecer faixas de valores e indicar, para cada registro, a que faixa de valor ele pertence.

> **Nota:** Este procedimento de transformar uma variável quantitativa em qualitativa também é conhecido como 
> discretização de variável.

Para ilustrar a execução deste procedimento vamos utilizar o conjunto de dados `Invoices.csv`.

O primeiro passo para se definir os estratos consiste em se estabelecer as faixas de valores para a variável a ser 
utilizada na estratificação nas quais se deseja classificar os registros. Para isso, faz-se necessário conhecer os 
valores máximo e mínimo do conjunto de dados, o que pode ser feito facilmente com a funções `min()` e `max()`.

In [3]:
print(min(faturas.Amount))

print(max(faturas.Amount))

5.0
13438.5


Considerando que os valores das faturas variam entre `R\$ 5,00` e `R\$ 13.438,50` e que desejamos estabelecer 3 estratos 
(baixos valores, valores medianos e altos valores) poderíamos ter os seguintes estratos:

**Estrato 1** - `R\$ 5,00` a `R\$1.000,00`    
**Estrato 2** - `R\$ 1.000,01` a `R\$10.000,00`   
**Estrato 3** - `R\$ 10.000,01` a `R\$ 13.438,50`   

A definição da quantidade de estratos (3 neste exemplo) e da amplitude de cada faixa é uma escolha subjetiva do auditor.  Deve-se observar que as faixas de valores podem ou não ter a mesma amplitude.

Para realizar a classificação dos registros da base de dados em cada um dos estratos definidos utiliza-sa a função 
**`cut()`**, como exemplificado a seguir:

In [4]:
faturas = (faturas
           .assign(Estratos = pd.cut(faturas.Amount,
                                     bins=[0.00, 1000.00, 10000.00, 13500.00],
                                     labels=["Baixo", "Medio", "Alto"])))

faturas.head()

,Date,InvoiceNo,CustomerNo,SalesPerson,ProductNo,UnitPrice,Quantity,Amount,Estratos
0,09/07/2003,20000,10220,8,8,9.2,41,377.2,Baixo
1,21/08/2003,20001,10491,4,48,14.0,30,420.0,Baixo
2,27/08/2003,20002,10704,3,43,15.0,25,375.0,Baixo
3,28/05/2003,20003,10430,5,54,24.0,22,528.0,Baixo
4,06/12/2003,20004,10841,17,11,15.0,21,315.0,Baixo


Agora o conjunto de dados contém uma nova coluna (`Estratos`) que classifica cada observação em um dos 3 estratos 
definidos pelo auditor.

Feita a estratificação, pode-se agora calcular a quantidade de registros em cada estrato, o valor médio de cada estrato, 
a percentagem que cada estrato representa do valor total, etc.  Este é o assunto do tópico 
[Sumarização](https://colab.research.google.com/github/brunomssmelo/Python_em_Auditoria/blob/master/06_Estratificacao.ipynb).

Para dividir a base de dados em `n` estratos de igual amplitude poderíamos utilizar o argumento `bins=` da função 
`cut()` definida na biblioteca *pandas*.

Supondo que quiséssemos dividir a base de dados em 10 estratos de mesma amplitude, poderíamos fazer da seguinte forma:

In [5]:
faturas = (faturas
           .assign(Estratos2 = pd.cut(faturas.Amount, bins=10, precision=5)))

faturas.head()

,Date,InvoiceNo,CustomerNo,SalesPerson,ProductNo,UnitPrice,Quantity,Amount,Estratos,Estratos2
0,09/07/2003,20000,10220,8,8,9.2,41,377.2,Baixo,"(-8.4335, 1348.35]"
1,21/08/2003,20001,10491,4,48,14.0,30,420.0,Baixo,"(-8.4335, 1348.35]"
2,27/08/2003,20002,10704,3,43,15.0,25,375.0,Baixo,"(-8.4335, 1348.35]"
3,28/05/2003,20003,10430,5,54,24.0,22,528.0,Baixo,"(-8.4335, 1348.35]"
4,06/12/2003,20004,10841,17,11,15.0,21,315.0,Baixo,"(-8.4335, 1348.35]"


Os estrados definidos podem ser consultados da seguinte forma:

In [6]:
faturas.Estratos2.cat.categories

IntervalIndex([(-8.4335, 1348.35], (1348.35, 2691.7], (2691.7, 4035.05], (4035.05, 5378.4], (5378.4, 6721.75], (6721.75, 8065.1], (8065.1, 9408.45], (9408.45, 10751.8], (10751.8, 12095.15], (12095.15, 13438.5]],
              closed='right',
              dtype='interval[float64]')

> **Nota:** Como não foi fornececido nomes para os estratos, a exemplo do que foi feito anteriormente, a função `cut()` 
> retorna os estratos no formato de intervalos abertos à esquerda e fechados à direita.

> **Nota:** O argumento `precision=` foi utilizado para evitar que os limites dos intervalos fossem fornecidos em notação cientifica. 
> Este argumento fornece a quantidade de dígitos a ser utilizada para elaborar os limites dos intervalos.

<br>

---

![](./imagens/sua_vez_fundo_transp.png) **Agora é com você...**

---



<br>

**1** - Crie uma nova variável que classifique os preços unitários dos produtos (`UnitPrice`) em 4 classes. Defina as 
classses da forma que entender mais conveniente. Coloque *labels* adequados.

<br>

**2** - Faça o mesmo com a variável `Quantity`, mas defina 3 classes de igual amplitude.